In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
%matplotlib inline

In [2]:
buisness = pd.read_csv('../input/yelp-csv/yelp_academic_dataset_business.csv')
review = pd.read_csv('../input/yelp-csv/yelp_academic_dataset_review.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,5,8,15,18,21,27,30,43,49,52,62,64,66,70,84,89,92,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
buisness_n = buisness[buisness['categories'].str.contains('Restaurant') == True]

In [4]:
buisness_n = buisness_n.fillna(0)

In [5]:
review_n = review[review.business_id.isin(buisness_n['business_id']) == True]

In [6]:
review_new = review_n.sample(n = 350000, random_state = 42)

In [7]:
text = review_new['text']

In [8]:
texts_n = []
for i in text:
    i = i.replace("\n", " ")
    texts_n.append(i)

In [9]:
from nltk.tokenize import word_tokenize
token_texts = []
for i in range (len(texts_n)):
    r = word_tokenize(texts_n[i])
    token_texts.append(r)

In [10]:
from string import punctuation
texts_new = [" ".join([word for word in text if word not in punctuation and not word.isnumeric() \
                      and len(word) > 1]) for text in token_texts]
texts_new[1]

"we went here last night with group of friends and got beers at the bar while waiting for table the beer selection is n't the greatest but it 's pretty solid we enjoyed some potosi cave ale and were seated in about minutes not too bad for group of on saturday night especially considering there was huge line of people waiting had the bad breath burger which truly lived up to its name the beef was quality and all the toppings on it were garlicky cheesy awesome goodness the bf had the reuben and said it was pretty tasty too think it 's safe to say we 'll be coming back"

In [13]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
lem_texts = []
token_texts2 = []
for i in range (len(texts_new)):
    r = word_tokenize(texts_new[i])
    token_texts2.append(r)
for text in token_texts2:
    l = ' '.join([lem.lemmatize(w) for w in text])
    lem_texts.append(l)
lem_texts[0]

"mean who doe n't love walking into Fred 's and eating lunch in the middle of Barneys ... 've been twice and honestly had great experience amazing service and delicious food Once for brunch had the prix fix brunch that includes salad brunch entree drink dessert AND coffee ... for buck not bad deal ... second time came wa little different strolled in with friend and we had boozy weekday lunch he had the burger and had the chicken salad ... he ordered manhattan which he exclaimed wa the best one he 's ever had and had delicious pinot grigio micah MISSSSS you but the wait staff wa the best we made friend with them and chatted them up for almost two hour ... they kept coming over and gave u cute place to go too ... they were really sweet Too bad it INSANELY overpriced like the Proenza Schoulers"

In [21]:
review = review.drop(['text'], axis=1)

In [22]:
review['text'] = lem_texts

In [23]:
review = review_new[['text','stars']]

In [24]:
train = review[0:280000]
test = review[280000:]

In [25]:
train = pd.get_dummies(train, columns = ['stars'])

In [26]:
test = pd.get_dummies(test, columns = ['stars'])

In [27]:
train = train.sample(frac = 0.1, random_state = 42)
test = test.sample(frac = 0.1, random_state = 42)

In [28]:
class_names = ['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']
y_train = train[class_names].values

In [29]:
w2v_model = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt'

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
w2v_index = dict(get_coefs(*o.strip().split()) for o in open(w2v_model))

In [30]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(train['text'].values))


Using TensorFlow backend.


In [31]:
X_train = tokenizer.texts_to_sequences(train['text'].values)
X_test = tokenizer.texts_to_sequences(test['text'].values)
X_train = pad_sequences(X_train, maxlen = 200)
X_test = pad_sequences(X_test, maxlen = 200)

In [32]:
word_index = tokenizer.word_index

In [34]:
nb_words = min(20000, len(word_index))
w2v_matrix = np.zeros((nb_words, 200))
missed = []
for word, i in word_index.items():
    if i >= 20000: break
    w2v_vector = w2v_index.get(word)
    if w2v_vector is not None:
        w2v_matrix[i] = w2v_vector
    else:
        missed.append(word)

In [35]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
embed_size = 200 
max_features = 20000
maxlen = 200

inp = Input(shape = (maxlen,))
x = Embedding(max_features, embed_size, weights = [w2v_matrix], trainable = True)(inp)
x = SpatialDropout1D(0.5)(x)
x = Bidirectional(LSTM(40, return_sequences=True))(x)
x = Bidirectional(GRU(40, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(5, activation = 'sigmoid')(conc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [36]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model = Model(inputs = inp, outputs = outp)
# patience is how many epochs to wait to see if val_loss will improve again.
earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = 'yelp_lstm_gru_weights.hdf5')
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [37]:
model.fit(X_train, y_train, batch_size = 512, epochs = 20, validation_split = .1, callbacks=[earlystop, checkpoint])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 25200 samples, validate on 2800 samples
Epoch 1/20
25200/25200 [==============================] - 62s 2ms/step - loss: 0.4961 - acc: 0.7833 - val_loss: 0.4639 - val_acc: 0.8001
Epoch 2/20
25200/25200 [==============================] - 59s 2ms/step - loss: 0.4430 - acc: 0.8048 - val_loss: 0.4167 - val_acc: 0.8104
Epoch 3/20
25200/25200 [==============================] - 58s 2ms/step - loss: 0.4002 - acc: 0.8153 - val_loss: 0.3841 - val_acc: 0.8175
Epoch 4/20
25200/25200 [==============================] - 57s 2ms/step - loss: 0.3769 - acc: 0.8222 - val_loss: 0.3614 - val_acc: 0.8288
Epoch 5/20
25200/25200 [==============================] - 58s 2ms/step - loss: 0.3577 - acc: 0.8294 - val_loss: 0.3486 - val_acc: 0.8345
Epoch 6/20
25200/25200 [==============================] - 58s 2ms/step - loss: 0.3465 - acc: 0.8347 - val_loss: 0.3444 - val_acc: 0.8338
Epo

In [38]:
pred = model.predict([X_test], batch_size=1024, verbose = 1)

7000/7000 [==============================] - 3s 498us/step


In [39]:
model.evaluate(X_test, test[class_names].values, verbose = 1, batch_size=1024)

7000/7000 [==============================] - 3s 411us/step


[0.3293969285828727, 0.841542843750545]

In [ ]:
review_new.stars.hist()